<a href="https://colab.research.google.com/github/mohammadbadi/Clustering_Frequency/blob/main/Code%20Sections/5.8%20Summary%20Table%20of%20Clustering%20Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **5.8 Summary Table of Clustering Models**

In [1]:
import warnings                                                                   # Import necessary libraries
import pandas as pd
import asyncio
!pip install dataframe_image -qqq
import dataframe_image as dfi
from IPython.display import display, HTML
from google.colab import files

print("\n\n")
warnings.filterwarnings("ignore", category=DeprecationWarning)                    # Ignore Deprecation Warnings
warnings.filterwarnings("ignore", category=FutureWarning)                         # Ignore future warnings

# Read the CSV file with updated feature names
url = "https://raw.githubusercontent.com/mohammadbadi/Clustering_Frequency/refs/heads/main/Output_CSV/Feature_Combo_New_Results.csv"  # Load feature combinations
data = pd.read_csv(url)

metrics = [                                                                       # Define the metrics to evaluate
    'KMeans Silhouette Score',
    'KMeans Calinski-Harabasz Score',
    'DBSCAN Silhouette Score',
    'KMeans Davies-Bouldin Index',
    'DBSCAN Davies-Bouldin Index'
]

top_results = {}                                                                  # Initialize a dictionary to store top results and their counts
feature_set_summary = {}

for metric in metrics:                                                            # Iterate through the top 100 rows for each metric and store occurrences
    if metric in data.columns:
        if 'Davies-Bouldin' in metric:                                            # For Davies-Bouldin Index, we want the lowest values
            top_rows = data.nsmallest(100, metric)
        else:
            top_rows = data.nlargest(100, metric)
        top_results[metric] = top_rows                                            # Store top rows in a dictionary
        for _, row in top_rows.iterrows():                                        # Iterate through the top rows to count occurrences of Feature Sets and track where they were found
            feature_set = row['Features']  # Updated column name
            if feature_set not in feature_set_summary:
                feature_set_summary[feature_set] = {
                    'Count': 0,
                    'Found In': []
                }
            feature_set_summary[feature_set]['Count'] += 1
            feature_set_summary[feature_set]['Found In'].append(metric)

summary_df = pd.DataFrame.from_dict(feature_set_summary, orient='index')          # Create a summary of most repeated feature sets
summary_df.reset_index(inplace=True)
summary_df.columns = ['Features', 'Count', 'Found In']                            # Updated column name
summary_df.sort_values(by='Count', ascending=False, inplace=True)                 # Sort by Count in descending order
final_metrics_df = pd.DataFrame(columns=['Set Number','Features', 'Count', *metrics])          # Prepare the final metrics DataFrame

for index, row in summary_df.iterrows():
    feature_set_name = row['Features']
    metrics_row = data[data['Features'] == feature_set_name]
    if not metrics_row.empty:
        new_row = {
            'Set Number': metrics_row['Set Number'].values[0],  # Fetch Set Number from CSV
            'Features': feature_set_name,
            'Count': row['Count'],
            **{metric: metrics_row[metric].values[0] for metric in metrics}
        }

        if final_metrics_df.empty:                                                # Handle empty DataFrame scenario properly before concatenation
            final_metrics_df = pd.DataFrame([new_row])                            # Append the new row to the final metrics DataFrame
        else:
            final_metrics_df = pd.concat([final_metrics_df, pd.DataFrame([new_row])], ignore_index=True)

final_metrics_df.sort_values(by='Count', ascending=False, inplace=True)           # Sort the final metrics DataFrame by Count

# Define a helper function to highlight the top 5 values correctly
def highlight_top5(s, ascending=True):
    # Sort the series and drop NaNs
    sorted_vals = s.sort_values(ascending=ascending).dropna()
    # Get the top 5 unique values
    top5_vals = sorted_vals.head(5).unique()
    # Return styling only for values in top5_vals
    return ['background-color: lightgreen' if x in top5_vals else '' for x in s]

# Apply highlighting with correct order:
# For silhouette and Calinski-Harabasz, higher is better (descending sort)
# For Davies-Bouldin Index, lower is better (ascending sort)
styled_table = final_metrics_df.style.apply(lambda s: highlight_top5(s, ascending=False), subset=['KMeans Silhouette Score']) \
                                      .apply(lambda s: highlight_top5(s, ascending=False), subset=['DBSCAN Silhouette Score']) \
                                      .apply(lambda s: highlight_top5(s, ascending=True), subset=['KMeans Davies-Bouldin Index']) \
                                      .apply(lambda s: highlight_top5(s, ascending=True), subset=['DBSCAN Davies-Bouldin Index']) \
                                      .apply(lambda s: highlight_top5(s, ascending=False), subset=['KMeans Calinski-Harabasz Score']) \
                                      .format({metric: '{:.2f}' for metric in metrics}) \
                                      .set_table_styles([
                                          {'selector': 'th', 'props': [('background-color', '#4CAF50'),
                                                                      ('color', 'white'),
                                                                      ('font-weight', 'bold'),
                                                                      ('text-align', 'center'),
                                                                      ('border', '1px solid black')]},  # Center-align and add black borders to headers
                                          {'selector': 'td', 'props': [('padding', '10px'),
                                                                      ('text-align', 'center'),
                                                                      ('border', '1px solid black')]},  # Center-align and add black borders to all cells
                                          {'selector': 'table', 'props': [('border-collapse', 'collapse')]}  # Ensure border styling applies correctly
                                      ]).set_properties(**{'border': '1px solid black'}) \
                                      .set_caption("<h3 style='color: navy; text-align: center;'>📊 Metrics Summary Table</h3>")

display(HTML("<h2 style='color: green; font-size: 20px;'><b> Styled Metrics Summary Table:</b></h2>"))  # Display styled table with formatted title
display(styled_table)

styled_table.data.to_excel('metrics_summary_table.xlsx', index=False)             # Save the styled table to an Excel file

display(HTML("""
    <p style="color: darkblue; font-size: 18px; font-weight: bold;">
         Metrics summary table has been saved to <span style="color: green;">'metrics_summary_table.xlsx'</span>.
    </p>
"""))
dfi.export(styled_table.data, 'metrics_summary_table.png', table_conversion='matplotlib', max_rows=-1)
files.download('metrics_summary_table.png')                                       # Download the results in PNG file
files.download('metrics_summary_table.xlsx')                                      # Download the results in Excel File


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 10.2 MB/s eta 0:00:00





,Set Number,Features,Count,KMeans Silhouette Score,KMeans Calinski-Harabasz Score,DBSCAN Silhouette Score,KMeans Davies-Bouldin Index,DBSCAN Davies-Bouldin Index
0,45,"['DOW_Weekend', 'DOW_Begin', 'DOW_Mid', 'DIV_HOOD_Hier']",5,0.87,24084.96,0.91,0.47,0.11
1,41,"['DOW_Weekend', 'DOW_Begin', 'DOW_Mid', 'HOOD_Freq']",5,0.86,22497.95,0.89,0.47,0.12
2,34,"['DOW_Weekend', 'DOW_Begin', 'DOW_Mid', 'LOCATION_Freq']",5,0.76,10055.31,0.95,0.60,0.06
3,36,"['DOW_Weekend', 'DOW_Begin', 'DOW_Mid', 'Loca_Premi_Freq']",5,0.74,9566.88,0.94,0.62,0.09
4,35,"['DOW_Weekend', 'DOW_Begin', 'DOW_Mid', 'PREMISES_Freq']",5,0.73,8753.35,0.99,0.49,0.04
5,37,"['DOW_Weekend', 'DOW_Begin', 'DOW_Mid', 'Division_Freq']",5,0.67,9700.03,0.77,0.50,0.36
6,593,"['OCC_HOUR', 'DOW_Weekend', 'DOW_Begin', 'DOW_Mid']",5,0.66,7697.93,0.66,0.48,0.59
7,145,"['OCC_YEAR', 'DOW_Weekend', 'DOW_Begin', 'DOW_Mid']",5,0.66,9375.40,0.67,0.51,0.56
8,42,"['DOW_Weekend', 'DOW_Begin', 'DOW_Mid', 'HOOD_Freq', 'LOCATION_Freq']",5,0.65,5168.01,0.76,0.69,0.22
9,48,"['DOW_Weekend', 'DOW_Begin', 'DOW_Mid', 'DIV_HOOD_Hier', 'Loca_Premi_Freq']",5,0.65,5280.07,0.75,0.67,0.23


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>